# CityBikes

In [1]:
import pandas as pd
import requests
import json

Send a request to CityBikes for the city of your choice. 

In [2]:
#Getting response for all cities to find href for Vancouver
response = requests.request("GET", url="http://api.citybik.es/v2/networks")
city_bikes = response.json()

In [3]:
#Looping through the response to get Vancouver bikes info
vancouver = None
for i in city_bikes["networks"]:
    if i["location"]["city"].lower() == "vancouver":
        vancouver = i
        break

In [4]:
vancouver

{'company': ['Vanncouver Bike Share Inc.',
  'CycleHop LLC',
  'City of Vancouver',
  'Shaw Communications Inc.',
  'Fifteen'],
 'gbfs_href': 'https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json',
 'href': '/v2/networks/mobibikes',
 'id': 'mobibikes',
 'location': {'city': 'Vancouver',
  'country': 'CA',
  'latitude': 49.2827,
  'longitude': -123.1207},
 'name': 'Mobi'}

In [5]:
#Getting response for Vancouver bikes
response_van = requests.request("GET", url="http://api.citybik.es/v2/networks/mobibikes")
data_van = response_van.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [6]:
#Looking into the response structure to figure what and how to get it to my df later
stations_van = data_van["network"]["stations"]
stations_van[:3]

[{'empty_slots': 32,
  'extra': {'ebikes': 0,
   'has_ebikes': True,
   'last_updated': 1723090537,
   'normal_bikes': 1,
   'renting': True,
   'returning': True,
   'slots': 35,
   'uid': '0001'},
  'free_bikes': 1,
  'id': '7a19c49f486d7c0c02b3685d7b240448',
  'latitude': 49.262487,
  'longitude': -123.114397,
  'name': '10th & Cambie',
  'timestamp': '2024-08-08T04:21:41.969000Z'},
 {'empty_slots': 5,
  'extra': {'ebikes': 1,
   'has_ebikes': True,
   'last_updated': 1723090823,
   'normal_bikes': 9,
   'renting': True,
   'returning': True,
   'slots': 16,
   'uid': '0004'},
  'free_bikes': 10,
  'id': '32603a87cfca71d0f7dfa3513bad69d5',
  'latitude': 49.274566,
  'longitude': -123.121817,
  'name': 'Yaletown-Roundhouse Station',
  'timestamp': '2024-08-08T04:21:41.924000Z'},
 {'empty_slots': 16,
  'extra': {'ebikes': 1,
   'has_ebikes': True,
   'last_updated': 1723090445,
   'normal_bikes': 7,
   'renting': True,
   'returning': True,
   'slots': 26,
   'uid': '0005'},
  'free_b

In [7]:
#Creating a function to parse the json response and get only those info I need
def parse_json(data):
    bike_stations = []
    for i in data:
        bike_stations.append({
        "uid":          i["extra"]["uid"],
        "latitude":     i["latitude"],
        "longitude":    i["longitude"],
        "free_bikes":   i["free_bikes"],
        "empty_slots":  i["empty_slots"],
        "station_name": i["name"]
        })
    return bike_stations

In [8]:
vancouver_stations = parse_json(stations_van)

Put your parsed results into a DataFrame.

In [9]:
#Putting the obtained earlier dictionary into data frame
stations_van_df = pd.DataFrame(vancouver_stations)
stations_van_df

,uid,latitude,longitude,free_bikes,empty_slots,station_name
0,0001,49.262487,-123.114397,1,32,10th & Cambie
1,0004,49.274566,-123.121817,10,5,Yaletown-Roundhouse Station
2,0005,49.279764,-123.110154,8,16,Dunsmuir & Beatty
3,0007,49.260599,-123.113504,3,12,12th & Yukon (City Hall)
4,0008,49.264215,-123.117772,3,12,8th & Ash
...,...,...,...,...,...,...
249,0617,49.277510,-123.061429,5,14,Adanac & Lakewood
250,0013,49.283482,-123.103738,15,10,Maple Tree Square
251,0400,49.271221,-123.191133,17,7,Jericho Beach
252,0169,49.283668,-123.109002,11,5,Cambie & Cordova


In [10]:
#Saving the data to csv
stations_van_df.to_csv("vancouver_bikes.csv", index = False)